In [1]:
import numpy as np
import pandas as pd
import string 
import random
import re

In [2]:
with open('businessCard_IBO.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()


In [3]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))
print(data)

[['id', 'text', 'tag'], ['000.jpeg', ' ', 'O'], ['000.jpeg', '.', 'O'], ['000.jpeg', '040-4852', 'B-PHONE'], ['000.jpeg', '"8881,"', 'I-PHONE'], ['000.jpeg', '90309', 'B-PHONE'], ['000.jpeg', '52549', 'I-PHONE'], ['000.jpeg', 'Fi', 'O'], ['000.jpeg', '/laurelsoverseaseducation', 'O'], ['000.jpeg', '@:', 'O'], ['000.jpeg', 'LAURELS', 'B-ORG'], ['000.jpeg', 'OVERSEAS', 'I-ORG'], ['000.jpeg', 'EDUCATIONAL', 'I-ORG'], ['000.jpeg', 'CONSULTANCY', 'I-ORG'], ['000.jpeg', 'PVT.', 'I-ORG'], ['000.jpeg', 'LTD.', 'I-ORG'], ['000.jpeg', 'Sea', 'O'], ['000.jpeg', '|', 'O'], ['000.jpeg', 'U.K', 'O'], ['000.jpeg', 'AUSTRALIA', 'O'], ['000.jpeg', 'CANADA', 'O'], ['000.jpeg', 'IRELAND', 'O'], ['000.jpeg', ' ', 'O'], ['000.jpeg', ' ', 'O'], ['000.jpeg', ' ', 'O'], ['000.jpeg', ' ', 'O'], ['000.jpeg', ' ', 'O'], ['000.jpeg', ' ', 'O'], ['000.jpeg', 'www.laurelseducation.com', 'B-WEB'], ['000.jpeg', ')%info@laurelseducation.com', 'B-EMAIL'], ['000.jpeg', ' ', 'O'], ['001.jpeg', 'john', 'B-NAME'], ['001.jp

In [4]:
df = pd.DataFrame(data[1:],columns=data[0])
df.head(10)

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,Fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@:,O
9,000.jpeg,LAURELS,B-ORG


Cleaning Data

In [5]:
#Removing White Spaces and unwanted special characters
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [6]:
df['text'] = df['text'].apply(cleanText)
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_10208\97840277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [7]:
dataClean

,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
...,...,...,...
10437,290.jpeg,p,O
10439,290.jpeg,srmrrisurres,B-ORG
10441,290.jpeg,richard,B-NAME
10442,290.jpeg,pretorius,I-NAME


Convert Data into Spacy Format

In [8]:
group = dataClean.groupby('id')
group.groups.keys()


dict_keys(['000.jpeg', '001.jpeg', '002.jpeg', '003.jpeg', '004.jpeg', '007.jpeg', '008.jpeg', '009.jpeg', '010.jpeg', '011.jpeg', '012.jpeg', '013.jpeg', '014.jpeg', '015.jpeg', '016.jpeg', '017.jpeg', '018.jpeg', '020.jpeg', '021.jpeg', '022.jpeg', '023.jpeg', '024.jpeg', '025.jpeg', '027.jpeg', '028.jpeg', '030.jpeg', '031.jpeg', '032.jpeg', '033.jpeg', '034.jpeg', '035.jpeg', '036.jpeg', '037.jpeg', '038.jpeg', '039.jpeg', '040.jpeg', '041.jpeg', '042.jpeg', '043.jpeg', '044.jpeg', '045.jpeg', '047.jpeg', '048.jpeg', '049.jpeg', '050.jpeg', '051.jpeg', '052.jpeg', '053.jpeg', '054.jpeg', '055.jpeg', '056.jpeg', '057.jpeg', '058.jpeg', '059.jpeg', '060.jpeg', '061.jpeg', '062.jpeg', '063.jpeg', '064.jpeg', '065.jpeg', '066.jpeg', '067.jpeg', '068.jpeg', '069.jpeg', '070.jpeg', '071.jpeg', '072.jpeg', '073.jpeg', '074.jpeg', '075.jpeg', '076.jpeg', '078.jpeg', '079.jpeg', '080.jpeg', '081.jpeg', '082.jpeg', '083.jpeg', '084.jpeg', '085.jpeg', '086.jpeg', '087.jpeg', '088.jpeg', '089.

In [9]:
cards = group.groups.keys()
all_cards_data = []

for card in cards:
    card_data = []
    group_array = group.get_group(card)[['text', 'tag']].values
    # print(group_array)
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in group_array:
        text = str(text)
        length = len(text) + 1   # 1 indiactes the space after each word
        start = end
        end = start + length

        if label != 'O':
            annotations['entities'].append((start, end-1, label))

        content += text + ' '

    card_data = (content, annotations)
    all_cards_data.append(card_data)


In [10]:
all_cards_data

[('. 040-4852 "8881," 90309 52549 fi /laurelsoverseaseducation @ laurels overseas educational consultancy pvt. ltd. sea u.k australia canada ireland www.laurelseducation.com info@laurelseducation.com ',
  {'entities': [(2, 10, 'B-PHONE'),
    (11, 18, 'I-PHONE'),
    (19, 24, 'B-PHONE'),
    (25, 30, 'I-PHONE'),
    (62, 69, 'B-ORG'),
    (70, 78, 'I-ORG'),
    (79, 90, 'I-ORG'),
    (91, 102, 'I-ORG'),
    (103, 107, 'I-ORG'),
    (108, 112, 'I-ORG'),
    (146, 170, 'B-WEB'),
    (171, 196, 'B-EMAIL')]}),
 ('john smith marketing manager web www.psdgraphics.com phone 123-456-7890 mail email@psdgraphics.com ',
  {'entities': [(0, 4, 'B-NAME'),
    (5, 10, 'I-NAME'),
    (11, 20, 'B-DES'),
    (21, 28, 'I-DES'),
    (33, 52, 'B-WEB'),
    (59, 71, 'B-PHONE'),
    (77, 98, 'B-EMAIL')]}),
 ('sau 0 98489 24441 dy "08672," 224441 /enkateswapa wie ',
  {'entities': [(6, 11, 'B-PHONE'), (12, 17, 'I-PHONE'), (37, 49, 'B-ORG')]}),
 ('prasad @ "9,96,31,73,53,59,49,04,00,000" i flex design album d

Split The Data into Training and Testing


In [13]:
random.shuffle(all_cards_data)
n = len(all_cards_data)

train_data = all_cards_data[:int(n*0.9)]
test_data = all_cards_data[int(n*0.9):]


In [ ]:
import pickle

pickle.dump(train_data, open('data/Train_data.pickle', mode='wb'))
pickle.dump(test_data, open('data/Test_data.pickle', mode='wb'))
